In [4]:
# Imports
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

In [5]:
# Read the applicants_data.csv file from the Resources folder into a Pandas DataFrame
applicant_data_df = pd.read_csv("Resources/applicants_data.csv")

# Review the DataFrame
applicant_data_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [6]:
# Review the data types associated with the columns
applicant_data_df.dtypes

EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [7]:
# Drop the 'EIN' and 'NAME' columns from the DataFrame
applicant_data_df = applicant_data_df.drop(columns=['EIN', 'NAME'])

# Review the DataFrame
applicant_data_df.head()


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [8]:
# Create a list of categorical variables 
categorical_variables =  list(applicant_data_df.dtypes[applicant_data_df.dtypes == "object"].index)

# Display the categorical variables list
categorical_variables


['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [9]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

In [10]:
# Encode the categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(applicant_data_df[categorical_variables])


In [11]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(encoded_data, 
                          columns=enc.get_feature_names(categorical_variables))


# Review the DataFrame
encoded_df.head()

/Users/owardlaw/miniforge3/envs/ml/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [12]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
numerical = applicant_data_df.drop(columns=categorical_variables)

encoded_df = pd.concat(
    
    [numerical, encoded_df], 
    axis=1
)
# Review the Dataframe
encoded_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
# Define the target set y using the IS_SUCCESSFUL column
y = encoded_df["IS_SUCCESSFUL"]

# Display a sample of y
y.head()


0    1
1    1
2    0
3    1
4    1
Name: IS_SUCCESSFUL, dtype: int64

In [14]:
# Define features set X by selecting all columns but IS_SUCCESSFUL
X = encoded_df.drop(columns=["IS_SUCCESSFUL"])


# Review the features DataFrame
X.head()

,STATUS,ASK_AMT,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [15]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [16]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train) 
X_test_scaled = X_scaler.transform(X_test)  


In [17]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

116

In [18]:
# Define the number of neurons in the output layer
number_output_neurons = 1

In [19]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 =  (number_input_features + 1) // 2

# Review the number hidden nodes in the first layer
hidden_nodes_layer1

58

In [20]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = (hidden_nodes_layer1 + 1) // 2

# Review the number hidden nodes in the second layer
hidden_nodes_layer2

29

In [21]:
# Create the Sequential model instance
nn =  Sequential()


Metal device set to: Apple M1 Pro


2022-03-10 11:02:34.309016: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-10 11:02:34.309214: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [22]:
# Add the first hidden layer
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))


In [23]:
# Add the second hidden layer
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))


In [24]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=number_output_neurons, activation="sigmoid"))


In [25]:
# Display the Sequential model summary
nn.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 58)                6786      
                                                                 
 dense_1 (Dense)             (None, 29)                1711      
                                                                 
 dense_2 (Dense)             (None, 1)                 30        
                                                                 
Total params: 8,527
Trainable params: 8,527
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [27]:
# Fit the model using 50 epochs and the training data
nn.fit(X_train_scaled, y_train, epochs=50)


2022-03-10 11:02:36.128359: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/50


2022-03-10 11:02:36.336152: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


804/804 [==============================] - 7s 6ms/step - loss: 0.5795 - accuracy: 0.7172
Epoch 2/50
804/804 [==============================] - 5s 6ms/step - loss: 0.5543 - accuracy: 0.7294
Epoch 3/50
804/804 [==============================] - 5s 6ms/step - loss: 0.5502 - accuracy: 0.7308
Epoch 4/50
804/804 [==============================] - 4s 6ms/step - loss: 0.5492 - accuracy: 0.7315
Epoch 5/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5471 - accuracy: 0.7333
Epoch 6/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5455 - accuracy: 0.7326
Epoch 7/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5440 - accuracy: 0.7343
Epoch 8/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5434 - accuracy: 0.7343
Epoch 9/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5432 - accuracy: 0.7358
Epoch 10/50
804/804 [==============================] - 5s 6ms/step - loss: 0.5422 - accuracy: 0.7338
Epoch 11/5

In [28]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2022-03-10 11:06:15.814322: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


268/268 - 1s - loss: 0.5560 - accuracy: 0.7301 - 876ms/epoch - 3ms/step
Loss: 0.5559500455856323, Accuracy: 0.7301457524299622


In [29]:
# Set the model's file path
file_path = (".../Resources/AlphabetSoup.h5")

# Export your model to a HDF5 file
nn.save(file_path)


### Alternative Model 1

In [31]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

116

In [32]:
# Define the number of neurons in the output layer
number_output_neurons_A1 = 1


In [33]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A1 = (number_input_features + number_output_neurons) // 2 


# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A1

58

In [34]:
# Create the Sequential model instance
nn_A1 = Sequential()

In [58]:
# First hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer1_A1, input_dim=number_input_features, activation="relu"))


# Output layer
nn_A1.add(Dense(units=1, activation="sigmoid"))



# Check the structure of the model
nn_A1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 58)                6786      
                                                                 
 dense_4 (Dense)             (None, 1)                 59        
                                                                 
 dense_11 (Dense)            (None, 58)                116       
                                                                 
 dense_12 (Dense)            (None, 1)                 59        
                                                                 
Total params: 7,020
Trainable params: 7,020
Non-trainable params: 0
_________________________________________________________________


In [59]:
# Compile the Sequential model
nn_A1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [39]:
# Fit the model using 50 epochs and the training data
fit_model_A1 = nn_A1.fit(X_train_scaled, y_train, epochs=50)


Epoch 1/50
  5/804 [..............................] - ETA: 10s - loss: 0.4986 - accuracy: 0.7750 

2022-03-10 11:16:13.256135: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


804/804 [==============================] - 8s 9ms/step - loss: 0.5396 - accuracy: 0.7365
Epoch 2/50
804/804 [==============================] - 6s 8ms/step - loss: 0.5385 - accuracy: 0.7373
Epoch 3/50
804/804 [==============================] - 7s 9ms/step - loss: 0.5384 - accuracy: 0.7365
Epoch 4/50
804/804 [==============================] - 6s 7ms/step - loss: 0.5389 - accuracy: 0.7373
Epoch 5/50
804/804 [==============================] - 7s 8ms/step - loss: 0.5375 - accuracy: 0.7375
Epoch 6/50
804/804 [==============================] - 8s 9ms/step - loss: 0.5380 - accuracy: 0.7373
Epoch 7/50
804/804 [==============================] - 7s 9ms/step - loss: 0.5378 - accuracy: 0.7364
Epoch 8/50
804/804 [==============================] - 7s 9ms/step - loss: 0.5374 - accuracy: 0.7367
Epoch 9/50
804/804 [==============================] - 7s 8ms/step - loss: 0.5383 - accuracy: 0.7359
Epoch 10/50
804/804 [==============================] - 7s 9ms/step - loss: 0.5379 - accuracy: 0.7373
Epoch 11/5

#### Alternative Model 2

In [61]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

116

In [62]:
# Define the number of neurons in the output layer
number_output_neurons_A2 = 1

In [63]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A2 = (number_input_features + number_output_neurons) // 2 


# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A2

58

In [64]:
# Create the Sequential model instance
nn_A2 = Sequential()

In [66]:
# First hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer1_A2, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer1_A2, activation="relu"))

# Output layer
nn_A2.add(Dense(units=number_output_neurons_A2, activation="sigmoid"))

# Check the structure of the model
nn_A2.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 58)                6786      
                                                                 
 dense_15 (Dense)            (None, 58)                3422      
                                                                 
 dense_16 (Dense)            (None, 58)                3422      
                                                                 
 dense_17 (Dense)            (None, 1)                 59        
                                                                 
Total params: 13,689
Trainable params: 13,689
Non-trainable params: 0
_________________________________________________________________


In [51]:
# Compile the model
nn_A2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [52]:
# Fit the model
fit_model = nn_A2.fit(X_train_scaled, y_train, epochs=100)


Epoch 1/100
  7/804 [..............................] - ETA: 6s - loss: 0.6783 - accuracy: 0.5268  

2022-03-10 11:23:14.629375: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


804/804 [==============================] - 7s 9ms/step - loss: 0.5723 - accuracy: 0.7211
Epoch 2/100
804/804 [==============================] - 7s 9ms/step - loss: 0.5528 - accuracy: 0.7297
Epoch 3/100
804/804 [==============================] - 7s 9ms/step - loss: 0.5489 - accuracy: 0.7322
Epoch 4/100
804/804 [==============================] - 7s 9ms/step - loss: 0.5477 - accuracy: 0.7330
Epoch 5/100
804/804 [==============================] - 7s 9ms/step - loss: 0.5453 - accuracy: 0.7324
Epoch 6/100
804/804 [==============================] - 7s 9ms/step - loss: 0.5450 - accuracy: 0.7339
Epoch 7/100
804/804 [==============================] - 7s 9ms/step - loss: 0.5433 - accuracy: 0.7342
Epoch 8/100
804/804 [==============================] - 7s 9ms/step - loss: 0.5427 - accuracy: 0.7352
Epoch 9/100
804/804 [==============================] - 6s 8ms/step - loss: 0.5421 - accuracy: 0.7350
Epoch 10/100
804/804 [==============================] - 7s 8ms/step - loss: 0.5414 - accuracy: 0.7356
E

In [53]:
print("Original Model Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Original Model Results
268/268 - 1s - loss: 0.5560 - accuracy: 0.7301 - 874ms/epoch - 3ms/step
Loss: 0.5559500455856323, Accuracy: 0.7301457524299622


In [54]:
print("Alternative Model 1 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A1.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 1 Results


2022-03-10 11:34:03.837949: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


268/268 - 1s - loss: 0.5783 - accuracy: 0.7304 - 929ms/epoch - 3ms/step
Loss: 0.5782617926597595, Accuracy: 0.7303789854049683


In [55]:
print("Alternative Model 2 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A2.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 2 Results


2022-03-10 11:34:04.802572: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


268/268 - 1s - loss: 0.5739 - accuracy: 0.7289 - 969ms/epoch - 4ms/step
Loss: 0.573871374130249, Accuracy: 0.7288629412651062


In [56]:
# Set the file path for the first alternative model
file_path = Path("./Resources/AlphabetSoup_A1.h5")

# Export your model to a HDF5 file
nn.save(file_path)

In [57]:
# Set the file path for the second alternative model
file_path = Path("./Resources/AlphabetSoup_A2.h5")

# Export your model to a HDF5 file
nn.save(file_path)
